The idea is to model the impact of fishing quotas on a fish population, while wanting to maximize food availability

Import all useful libraries

In [ ]:
#%load_ext autoreload
#%autoreload 2
import numpy as np
import copy 
import random
import pandas as pd
from tqdm import tqdm
from scipy.stats import binom
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [ ]:
rng = np.random.default_rng()

Import the q-learning function

In [ ]:
#from robust_q_learning_v3 import *
#from q_learning_v2 import *

In [ ]:
from robust_q_learning_v2 import *
from q_learning import *

First simplified case: one species, two size class

In [ ]:
X = np.array([(0,0), (0,5), (5,0), (5,5)]) # States
A = np.array([0, 5]) # Actions

c1 = 1 / 5

def r(x,a,y):
    x1, x2 = x
    y1, y2 = y
    return(c1 * (x2 - a >= 0) * a - 10 * (x2 - a < 0)) # Reward function

rr = 1
dr = 0
gr = 1
npp = 0
def P(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x1, x2 = x
        x2_ = max(x2 - a, 0)
        y1  = min(rr * x2_ + npp * 5, 5)
        y2  = min(x2_ - 5 * dr + gr * x1, 5)
        return(y1, y2)
    else:
        return rng.choice(X)
    
    
eps1 = 0.75
npp2 = 1
def P2(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x1, x2 = x
        unif2 = np.random.uniform(0)
        x2_ = max(x2 - a, 0)
        y1  = min(rr * x2_ + ((unif2 > eps1) * npp + (unif2 <= eps1) * npp2) * 5 , 5)
        y2  = min(x2_ - 5 * dr + gr * x1, 5)
        return(y1, y2)
    else:
        return rng.choice(X)

alpha      = 0.95  # Discount Factor
x_0        = (5,5) # Initial Value
eps_greedy = 0.1   # Epsilon greedy policy

In [ ]:
# Build the functions that allow us to get the index of an element a (reps. x) in A (resp. X)
if np.ndim(A) > 1:
    A_list = A
else:
    A_list = np.array([[a] for a in A])
if np.ndim(X) > 1:
    X_list = X
else:
    X_list = np.array([[x] for x in X])

def a_index(a):
    return np.flatnonzero((a==A_list).all(1))[0]
def x_index(x):
    return np.flatnonzero((x==X_list).all(1))[0]

First non-robust runs

In [ ]:
Nr_iter = 1_000_000

Q_0_, V = q_learning(X, A, r, P, alpha, x_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))

In [ ]:
# Get the result of the Q-Learning algorithm,
# Get the optimal results for each x in X
def a_opt(x, Q_opt):
    return A[np.argmax(Q_opt[x_index(x),:])]

In [ ]:
df = pd.DataFrame(np.array([[a_opt(x, Q_0_) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(5,0)', '(5,5)']
df

In [ ]:
V

Testing the accuracy of the result with the other function, and adding uncertainty

In [ ]:
X = np.array([(0,0), (0,5), (5,0), (5,5)]) # States
A = np.array([0, 5]) # Actions

c1 = 1 / 5

def r(x,a,y):
    x1, x2 = x
    y1, y2 = y
    return(c1 * (x2 - a >= 0) * a - (x2 - a < 0)) # Reward function

rr = 1
dr = 0
gr = 1
npp = 0
def P1(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x1, x2 = x
        x2_ = max(x2 - a, 0)
        y1  = min(rr * x2_ + npp * 5, 5)
        y2  = min(x2_ - 5 * dr + gr * x1, 5)
        return(y1, y2)
    else:
        return rng.choice(X)
    
    
eps1 = 1
npp2 = 1
def P2(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > eps1) * npp + (unif2 <= eps1) * npp2) * 5 , 5)
    y2  = min(x2_ - 5 * dr + gr * x1, 5)
    return(y1, y2)
    
# CREATE THE PROBABILITY MEASURE OUT OF THE RANDOM VARIABLE
nr = 1_000
p1_ = np.zeros([len(X), len(A), len(X)])
p2_ = np.zeros([len(X), len(A), len(X)])
for n in range(nr):
    for x in X:
        for a in A:
            y1 = P1(x,a)
            x_1 = x_index(y1)
            p1_[x_index(x), a_index(a), x_1] += 1
            y2 = P2(x,a)
            x_2 = x_index(y2)
            p2_[x_index(x), a_index(a), x_2] += 1
p1_ = p1_/nr
p2_ = p2_/nr
def p1(x,a,y):
    return(p1_[x_index(x), a_index(a), x_index(y)])
def p2(x,a,y):
    return(p2_[x_index(x), a_index(a), x_index(y)])

alpha      = 0.95  # Discount Factor
x_0        = (5,5) # Initial Value
k_0        = 0     # Initial index of the corresponding MDP
eps_greedy = 0.1   # Epsilon greedy policy

In [ ]:
Nr_iter = 1_000_000

Q_0_, V = q_learning(X, A, r, P2, alpha, x_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))

In [ ]:
df = pd.DataFrame(np.array([[a_opt(x, Q_0_) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(5,0)', '(5,5)']
df

In [ ]:
V

In [ ]:
Q_opt_robust, V = robust_q_learning_v2(X, A, r, np.array([P1, P2]), np.array([p1, p2]), alpha, x_0, k_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))

In [ ]:
df = pd.DataFrame(np.array([[a_opt(x, Q_opt_robust) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(5,0)', '(5,5)']
df

In [ ]:
V

In [ ]:
plt.matshow(V, cmap='gray')

Lets now consider more possible states (more size classes)

In [ ]:
#%load_ext autoreload
#%autoreload 2
import numpy as np
import copy 
import random
import pandas as pd
from tqdm import tqdm
from scipy.stats import binom
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [ ]:
rng = np.random.default_rng()

In [ ]:
#from robust_q_learning_v3 import *
#from q_learning_v2 import *

In [ ]:
from robust_q_learning_v2 import *
from q_learning import *

In [ ]:
X = np.array([(0,0), (0,5), (0,10), (5,0), (5,5), (5,10), (10,0), (10,5), (10,10)]) # States
A = np.array([0, 5, 10]) # Actions

In [ ]:
# Build the functions that allow us to get the index of an element a (reps. x) in A (resp. X)
if np.ndim(A) > 1:
    A_list = A
else:
    A_list = np.array([[a] for a in A])
if np.ndim(X) > 1:
    X_list = X
else:
    X_list = np.array([[x] for x in X])

def a_index(a):
    return np.flatnonzero((a==A_list).all(1))[0]
def x_index(x):
    return np.flatnonzero((x==X_list).all(1))[0]

In [ ]:
c1 = 1 / 10

def r(x,a,y):
    x1, x2 = x
    y1, y2 = y
    return(c1 * (x2 - a >= 0) * a - 10 * (x2 - a < 0)) # Reward function

eps_greedy = 0.1   # Epsilon greedy policy

rr = 1
dr = 0
gr = 1
npp = 0
def P(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x1, x2 = x
        x2_ = max(x2 - a, 0)
        y1  = min(rr * x2_ + npp * 5, 10)
        y2  = min(x2_ - 5 * dr + gr * x1, 10)
        return(y1, y2)
    else:
        return rng.choice(X)

# CREATE THE PROBABILITY MEASURE OUT OF THE RANDOM VARIABLE
nr = 1_000
p_ = np.zeros([len(X), len(A), len(X)])
for n in range(nr):
    for x in X:
        for a in A:
            y = P(x,a)
            x_ = x_index(y)
            p_[x_index(x), a_index(a), x_] += 1
p_ = p_/nr
def p(x,a,y):
    return(p_[x_index(x), a_index(a), x_index(y)])

alpha      = 0.95  # Discount Factor
x_0        = (10,10) # Initial Value
k_0        = 0     # Initial index of the corresponding MDP, starting with the central proba of 1/2

In [ ]:
Nr_iter = 1_000_000

Q_0_, V = q_learning(X, A, r, P, alpha, x_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))

In [ ]:
# Get the result of the Q-Learning algorithm,
# Get the optimal results for each x in X
def a_opt(x, Q_opt):
    return A[np.argmax(Q_opt[x_index(x),:])]

In [ ]:
df = pd.DataFrame(np.array([[a_opt(x, Q_0_) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(0,10)', '(5,0)', '(5,5)', '(5,10)', '(10,0)', '(10,5)', '(10,10)']
df

In [ ]:
V

In [ ]:
plt.matshow(V, cmap='gray')

Adding new transition kernels, and uncertainty

In [ ]:
c1 = 1 / 10

def r(x,a,y):
    x1, x2 = x
    y1, y2 = y
    return(c1 * (x2 - a >= 0) * a - 10 * (x2 - a < 0)) # Reward function

rr = 1
dr = 0
gr = 1
npp = 0
def P1(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x1, x2 = x
        x2_ = max(x2 - a, 0)
        y1  = min(rr * x2_ + npp * 5, 10)
        y2  = min(x2_ - 5 * dr + gr * x1, 10)
        return(y1, y2)
    else:
        return rng.choice(X)
    
    
eps1 = 0.9
npp2 = 1
def P2(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > eps1) * npp + (unif2 <= eps1) * npp2) * 5 , 10)
    y2  = min(x2_ - 5 * dr + gr * x1, 10)
    return(y1, y2)

eps3 = 0.5
dr3 = 1
rr3   = 2
def P3(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x1, x2 = x
        unif2 = np.random.uniform(0)
        x2_  = max(x2 - a, 0)
        y1   = min(rr3 * x2_ + npp * 5 , 10)
        x2__ = max(x2_ - ((unif2 > 0.5) * dr + (unif2 <= eps3) * 1) * 5, 0)
        y2   = min(x2__ + gr * x1, 10)
        return(y1, y2)
    else:
        return rng.choice(X)

    
# CREATE THE PROBABILITY MEASURE OUT OF THE RANDOM VARIABLE
nr = 1_000
p1_ = np.zeros([len(X), len(A), len(X)])
p2_ = np.zeros([len(X), len(A), len(X)])
p3_ = np.zeros([len(X), len(A), len(X)])
for n in range(nr):
    for x in X:
        for a in A:
            y1 = P1(x,a)
            x_1 = x_index(y1)
            p1_[x_index(x), a_index(a), x_1] += 1
            y2 = P2(x,a)
            x_2 = x_index(y2)
            p2_[x_index(x), a_index(a), x_2] += 1
            y3 = P3(x,a)
            x_3 = x_index(y3)
            p3_[x_index(x), a_index(a), x_3] += 1
p1_ = p1_/nr
p2_ = p2_/nr
p3_ = p3_/nr
def p1(x,a,y):
    return(p1_[x_index(x), a_index(a), x_index(y)])
def p2(x,a,y):
    return(p2_[x_index(x), a_index(a), x_index(y)])
def p3(x,a,y):
    return(p3_[x_index(x), a_index(a), x_index(y)])

alpha      = 0.95  # Discount Factor
x_0        = (10,10) # Initial Value
k_0        = 0     # Initial index of the corresponding MDP, starting with the central proba of 1/2
eps_greedy = 0.1   # Epsilon greedy policy

In [ ]:
Nr_iter = 1_000_000
Q_0_, V = q_learning(X, A, r, P3, alpha, x_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))
df = pd.DataFrame(np.array([[a_opt(x, Q_0_) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(0,10)', '(5,0)', '(5,5)', '(5,10)', '(10,0)', '(10,5)', '(10,10)']
df

In [ ]:
V

In [ ]:
# Defining all the 20 probabilities
def P4(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > 0.5) * npp + (unif2 <= 0.5) * 1) * 5 , 10)
    y2  = min(x2_ - 5 * dr + gr * x1, 10)
    return(y1, y2)

def P5(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > 0.25) * npp + (unif2 <= 0.25) * 1) * 5 , 10)
    y2  = min(x2_ - 5 * dr + gr * x1, 10)
    return(y1, y2)

def P6(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > 0.9) * npp + (unif2 <= 0.9) * 2) * 5 , 10)
    y2  = min(x2_ - 5 * dr + gr * x1, 10)
    return(y1, y2)

def P7(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > 0.5) * npp + (unif2 <= 0.5) * 2) * 5 , 10)
    y2  = min(x2_ - 5 * dr + gr * x1, 10)
    return(y1, y2)

def P8(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > 0.25) * npp + (unif2 <= 0.25) * 2) * 5 , 10)
    y2  = min(x2_ - 5 * dr + gr * x1, 10)
    return(y1, y2)

def P9(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x1, x2 = x
        unif2 = np.random.uniform(0)
        x2_  = max(x2 - a, 0)
        y1   = min(1 * x2_ + npp * 5 , 10)
        x2__ = max(x2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
        y2   = min(x2__ + gr * x1, 10)
        return(y1, y2)
    else:
        return rng.choice(X)

def P10(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x1, x2 = x
        unif2 = np.random.uniform(0)
        x2_  = max(x2 - a, 0)
        y1   = min(1 * x2_ + npp * 5 , 10)
        x2__ = max(x2_ - ((unif2 > 1) * dr + (unif2 <= 1) * 1) * 5, 0)
        y2   = min(x2__ + gr * x1, 10)
        return(y1, y2)
    else:
        return rng.choice(X)

def P11(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x1, x2 = x
        unif2 = np.random.uniform(0)
        x2_  = max(x2 - a, 0)
        y1   = min(2 * x2_ + npp * 5 , 10)
        x2__ = max(x2_ - ((unif2 > 1) * dr + (unif2 <= 1) * 1) * 5, 0)
        y2   = min(x2__ + gr * x1, 10)
        return(y1, y2)
    else:
        return rng.choice(X)

def P12(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > 0.9) * npp + (unif2 <= 0.9) * 1) * 5 , 10)
    x2__ = max(x2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
    y2  = min(x2__ + gr * x1, 10)
    return(y1, y2)

def P13(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > 0.5) * npp + (unif2 <= 0.5) * 1) * 5 , 10)
    x2__ = max(x2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
    y2  = min(x2__ + gr * x1, 10)
    return(y1, y2)

def P14(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(rr * x2_ + ((unif2 > 0.25) * npp + (unif2 <= 0.25) * 1) * 5 , 10)
    x2__ = max(x2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
    y2  = min(x2__ + gr * x1, 10)
    return(y1, y2)

def P15(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(2 * x2_ + ((unif2 > 0.9) * npp + (unif2 <= 0.9) * 1) * 5 , 10)
    x2__ = max(x2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
    y2  = min(x2__ + gr * x1, 10)
    return(y1, y2)

def P16(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(2 * x2_ + ((unif2 > 0.5) * npp + (unif2 <= 0.5) * 1) * 5 , 10)
    x2__ = max(x2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
    y2  = min(x2__ + gr * x1, 10)
    return(y1, y2)

def P17(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(2 * x2_ + ((unif2 > 0.25) * npp + (unif2 <= 0.25) * 1) * 5 , 10)
    x2__ = max(x2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
    y2  = min(x2__ + gr * x1, 10)
    return(y1, y2)

def P18(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(2 * x2_ + ((unif2 > 0.9) * npp + (unif2 <= 0.9) * 1) * 5 , 10)
    x2__ = max(x2_ - ((unif2 > 1) * dr + (unif2 <= 1) * 1) * 5, 0)
    y2  = min(x2__ + gr * x1, 10)
    return(y1, y2)

def P19(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(2 * x2_ + ((unif2 > 0.5) * npp + (unif2 <= 0.5) * 1) * 5 , 10)
    x2__ = max(x2_ - ((unif2 > 1) * dr + (unif2 <= 1) * 1) * 5, 0)
    y2  = min(x2__ + gr * x1, 10)
    return(y1, y2)

def P20(x,a):
    x1, x2 = x
    unif2 = np.random.uniform(0)
    x2_ = max(x2 - a, 0)
    y1  = min(2 * x2_ + ((unif2 > 0.25) * npp + (unif2 <= 0.25) * 1) * 5 , 10)
    x2__ = max(x2_ - ((unif2 > 1) * dr + (unif2 <= 1) * 1) * 5, 0)
    y2  = min(x2__ + gr * x1, 10)
    return(y1, y2)
    
# CREATE THE PROBABILITY MEASURE OUT OF THE RANDOM VARIABLE
nr = 1_000
p4_ = np.zeros([len(X), len(A), len(X)])
p5_ = np.zeros([len(X), len(A), len(X)])
p6_ = np.zeros([len(X), len(A), len(X)])
p7_ = np.zeros([len(X), len(A), len(X)])
p8_ = np.zeros([len(X), len(A), len(X)])
p9_ = np.zeros([len(X), len(A), len(X)])
p10_ = np.zeros([len(X), len(A), len(X)])
p11_ = np.zeros([len(X), len(A), len(X)])
p12_ = np.zeros([len(X), len(A), len(X)])
p13_ = np.zeros([len(X), len(A), len(X)])
p14_ = np.zeros([len(X), len(A), len(X)])
p15_ = np.zeros([len(X), len(A), len(X)])
p16_ = np.zeros([len(X), len(A), len(X)])
p17_ = np.zeros([len(X), len(A), len(X)])
p18_ = np.zeros([len(X), len(A), len(X)])
p19_ = np.zeros([len(X), len(A), len(X)])
p20_ = np.zeros([len(X), len(A), len(X)])
for n in range(nr):
    for x in X:
        for a in A:
            y4 = P4(x,a)
            x_4 = x_index(y)
            p4_[x_index(x), a_index(a), x_4] += 1
            y5 = P5(x,a)
            x_5 = x_index(y5)
            p5_[x_index(x), a_index(a), x_5] += 1
            y6 = P6(x,a)
            x_6 = x_index(y6)
            p6_[x_index(x), a_index(a), x_6] += 1
            y7 = P7(x,a)
            x_7 = x_index(y7)
            p7_[x_index(x), a_index(a), x_7] += 1
            y8 = P8(x,a)
            x_8 = x_index(y8)
            p8_[x_index(x), a_index(a), x_8] += 1
            y9 = P9(x,a)
            x_9 = x_index(y9)
            p9_[x_index(x), a_index(a), x_9] += 1
            y10 = P10(x,a)
            x_10 = x_index(y10)
            p10_[x_index(x), a_index(a), x_10] += 1
            y11 = P11(x,a)
            x_11 = x_index(y11)
            p11_[x_index(x), a_index(a), x_11] += 1
            y12 = P12(x,a)
            x_12 = x_index(y12)
            p12_[x_index(x), a_index(a), x_12] += 1
            y13 = P13(x,a)
            x_13 = x_index(y13)
            p13_[x_index(x), a_index(a), x_13] += 1
            y14 = P14(x,a)
            x_14 = x_index(y14)
            p14_[x_index(x), a_index(a), x_14] += 1
            y15 = P15(x,a)
            x_15 = x_index(y15)
            p15_[x_index(x), a_index(a), x_15] += 1
            y16 = P16(x,a)
            x_16 = x_index(y16)
            p16_[x_index(x), a_index(a), x_16] += 1
            y17 = P17(x,a)
            x_17 = x_index(y17)
            p17_[x_index(x), a_index(a), x_17] += 1
            y18 = P18(x,a)
            x_18 = x_index(y18)
            p18_[x_index(x), a_index(a), x_18] += 1
            y19 = P19(x,a)
            x_19 = x_index(y19)
            p19_[x_index(x), a_index(a), x_19] += 1
            y20 = P20(x,a)
            x_20 = x_index(y20)
            p20_[x_index(x), a_index(a), x_20] += 1
p4_ = p4_/nr
p5_ = p5_/nr
p6_ = p6_/nr
p7_ = p7_/nr
p8_ = p8_/nr
p9_ = p9_/nr
p10_ = p10_/nr
p11_ = p11_/nr
p12_ = p12_/nr
p13_ = p13_/nr
p14_ = p14_/nr
p15_ = p15_/nr
p16_ = p16_/nr
p17_ = p17_/nr
p18_ = p18_/nr
p19_ = p19_/nr
p20_ = p20_/nr
def p4(x,a,y):
    return(p4_[x_index(x), a_index(a), x_index(y)])
def p5(x,a,y):
    return(p5_[x_index(x), a_index(a), x_index(y)])
def p6(x,a,y):
    return(p6_[x_index(x), a_index(a), x_index(y)])
def p7(x,a,y):
    return(p7_[x_index(x), a_index(a), x_index(y)])
def p8(x,a,y):
    return(p8_[x_index(x), a_index(a), x_index(y)])
def p9(x,a,y):
    return(p9_[x_index(x), a_index(a), x_index(y)])
def p10(x,a,y):
    return(p10_[x_index(x), a_index(a), x_index(y)])
def p11(x,a,y):
    return(p11_[x_index(x), a_index(a), x_index(y)])
def p12(x,a,y):
    return(p12_[x_index(x), a_index(a), x_index(y)])
def p13(x,a,y):
    return(p13_[x_index(x), a_index(a), x_index(y)])
def p14(x,a,y):
    return(p14_[x_index(x), a_index(a), x_index(y)])
def p15(x,a,y):
    return(p15_[x_index(x), a_index(a), x_index(y)])
def p16(x,a,y):
    return(p16_[x_index(x), a_index(a), x_index(y)])
def p17(x,a,y):
    return(p17_[x_index(x), a_index(a), x_index(y)])
def p18(x,a,y):
    return(p18_[x_index(x), a_index(a), x_index(y)])
def p19(x,a,y):
    return(p19_[x_index(x), a_index(a), x_index(y)])
def p20(x,a,y):
    return(p20_[x_index(x), a_index(a), x_index(y)])

In [ ]:
Nr_iter = 1_000_000
Q_0_, V = q_learning(X, A, r, P16, alpha, x_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))
df = pd.DataFrame(np.array([[a_opt(x, Q_0_) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(0,10)', '(5,0)', '(5,5)', '(5,10)', '(10,0)', '(10,5)', '(10,10)']
df

In [ ]:
V

Now lets run the robust algorithm with more and more transition kernels

In [ ]:
Nr_iter = 1_000_000
Q_opt_robust, V = robust_q_learning_v2(X, A, r, np.array([P1, P2, P3]), np.array([p1, p2, p3]), alpha, x_0, k_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))
df = pd.DataFrame(np.array([[a_opt(x, Q_opt_robust) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(0,10)', '(5,0)', '(5,5)', '(5,10)', '(10,0)', '(10,5)', '(10,10)']
df

In [ ]:
Nr_iter = 1_000_000
Q_opt_robust, V = robust_q_learning_v2(X, A, r, np.array([P1, P3, P9, P10, P11]), np.array([p1, p3, p9, p10, p11]), alpha, x_0, k_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))
df = pd.DataFrame(np.array([[a_opt(x, Q_opt_robust) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(0,10)', '(5,0)', '(5,5)', '(5,10)', '(10,0)', '(10,5)', '(10,10)']
df

In [ ]:
V

In [ ]:
P_ = [P1, P2, P3, P4, P5, P6, P7, P8, P9, P10, P11, P12, P13, P14, P15, P16, P17, P18, P19, P20]
p_ = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10, p11, p12, p13, p14, p15, p16, p17, p18, p19, p20]
l = [j for j in range(len(P_))]
P = []
p = []
for i in range(40):
    J = random.choice(l)
    P += [P_[J]]
    p += [p_[J]]

In [ ]:
Nr_iter = 1_000_000
Q_opt_robust, V = robust_q_learning_v2(X, A, r, np.array(P), np.array(p), alpha, x_0, k_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))
df = pd.DataFrame(np.array([[a_opt(x, Q_opt_robust) for x in X]]))
df.columns = ['(0,0)', '(0,5)', '(0,10)', '(5,0)', '(5,5)', '(5,10)', '(10,0)', '(10,5)', '(10,10)']
df

In [ ]:
V

In [ ]:
Nb_kernel = [2, 3, 4, 5, 5, 10, 15, 15, 20, 30, 40, 50]
cv_time   = [182, 275, 369.3, 537, 443.8, 996, 1287.7, 1624.4, 1685, 2515.8, 3347.2, 4141.7]

plt.plot(Nb_kernel, cv_time, '*')
plt.xlabel("Number of kernels")
plt.ylabel("Convergence time (s)")

# Show the major grid lines with dark grey lines
plt.grid(which='major', color='#666666', linestyle='-', alpha = 0.5)

# Show the minor grid lines with very faint and almost transparent grey lines
plt.minorticks_on()
plt.grid(which='minor', color='#999999', linestyle='-', alpha=0.2)

plt.show()

Now lets add a new species to simulate the willingness to keep a certain biodiversity

In [ ]:
#%load_ext autoreload
#%autoreload 2
import numpy as np
import copy 
import random
import pandas as pd
from tqdm import tqdm
from scipy.stats import binom
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [ ]:
rng = np.random.default_rng()

In [ ]:
from robust_q_learning_v2 import *
from q_learning import *

In [ ]:
A_s1 = np.array([0, 5]) # Actions
A_s2 = np.array([0, 5, 10]) # Actions
X_s1    = []
X_s2    = []
for x1 in A_s1:
    for x2 in A_s1:
        X_s1 += [(x1, x2)]
X_s1 = np.array(X_s1) # States
for x1 in A_s2:
    for x2 in A_s2:
        X_s2 += [(x1, x2)]
X_s1 = np.array(X_s1) # States
X_s2 = np.array(X_s2) # States second species

A = []
for a1 in A_s1:
    for a2 in A_s2:
        A += [(a1, a2)]
A = np.array(A) # Actions
X = []
for x1 in X_s1:
    for x2 in X_s2:
        X += [(x1, x2)]
X = np.array(X) # States

In [ ]:
# Build the functions that allow us to get the index of an element a (reps. x) in A (resp. X)
if np.ndim(A) > 1:
    A_list = A
else:
    A_list = np.array([[a] for a in A])
if np.ndim(X) > 1:
    X_list = X
else:
    X_list = np.array([[x] for x in X])

def a_index(a):
    return np.flatnonzero((a==A_list).all(1))[0]
def x_index(x):
    return np.flatnonzero((x==X_list).all(1))[0]

In [ ]:
c1   = 2 / 5
c2   = 1 / 10 
psy  = 1

def r(x,a,y):
    x_s1, x_s2   = x
    y_s1, y_s2   = y
    x1_s1, x2_s1 = x_s1
    x1_s2, x2_s2 = x_s2
    y1_s1, y2_s1 = y_s1
    y1_s2, y2_s2 = y_s2
    a_s1, a_s2   = a

    food_s1      = c1 * (x2_s1 - a_s1 >= 0) * a_s1 - 10 * (x2_s1 - a_s1 < 0)  #Optimization term, regarding the food that the species 1 can give us
    food_s2      = c2 * (x2_s2 - a_s2 >= 0) * a_s2 - 10 * (x2_s2 - a_s2 < 0)  #Optimization term, regarding the food that the species 2 can give us
    biod         = psy  * (y1_s1 + y2_s1 > 0) * (y1_s2 + y2_s2 > 0)           #Optimization term, regarding the biodiversity we want to keep
    return(food_s1 + food_s2 + biod) # Reward function


eps_greedy = 0.1   # Epsilon greedy policy

rr = 1
dr = 0
gr = 1
npp = 0
def P(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x_s1, x_s2   = x
        x1_s1, x2_s1 = x_s1
        x1_s2, x2_s2 = x_s2
        a_s1, a_s2   = a 
        
        x2_s1_ = max(x2_s1 - a_s1, 0)
        y1_s1  = min(rr * x2_s1_ + npp * 5, 5)
        y2_s1  = min(x2_s1_ - 5 * dr + gr * x1_s1, 5)
        
        x2_s2_ = max(x2_s2 - a_s2, 0)
        y1_s2  = min(rr * x2_s2_ + npp * 5, 10)
        y2_s2  = min(x2_s2_ - 5 * dr + gr * x1_s2, 10)
        
        return ((y1_s1, y2_s1), (y1_s2, y2_s2))
    else:
        return rng.choice(X)

# CREATE THE PROBABILITY MEASURE OUT OF THE RANDOM VARIABLE
nr = 1_000
p_ = np.zeros([len(X), len(A), len(X)])
for n in range(nr):
    for x in X:
        for a in A:
            y = P(x,a)
            x_ = x_index(y)
            p_[x_index(x), a_index(a), x_] += 1
p_ = p_/nr
def p(x,a,y):
    return(p_[x_index(x), a_index(a), x_index(y)])

alpha      = 0.95  # Discount Factor
x_0        = ((5,5), (10, 10)) # Initial Value
k_0        = 0     # Initial index of the corresponding MDP, starting with the central proba of 1/2

In [ ]:
Nr_iter = 10_000_000

Q_0_, V = q_learning(X, A, r, P, alpha, x_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = np.ones([len(X),len(A)]))

In [ ]:
# Get the result of the Q-Learning algorithm,
# Get the optimal results for each x in X
def a_opt(x, Q_opt):
    return A[np.argmax(Q_opt[x_index(x),:])]

In [ ]:
Result = np.array([[a_opt(x, Q_0_) for x in X]])
Result = Result[0]

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:

df = pd.DataFrame(Result.T)
df.columns = pd.MultiIndex.from_tuples([(str(x[0]).replace("[", "(").replace("]", ")"), str(x[1]).replace("[", "(").replace("]", ")")) for x in X]) #[str(x) for x in X]
df.index.name = "Species"
df.columns.names = ["A", "B"]
df

In [ ]:
plt.matshow(V, cmap='gray')
plt.colorbar()

Closed system

In [ ]:
def P3(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x_s1, x_s2   = x
        x1_s1, x2_s1 = x_s1
        x1_s2, x2_s2 = x_s2
        a_s1, a_s2   = a 
        
        unif2 = np.random.uniform(0)

        x2_s1_ = max(x2_s1 - a_s1, 0)
        y1_s1  = min(2 * x2_s1_ + npp * 5, 5)
        x2_s1__ = max(x2_s1_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
        y2_s1  = min(x2_s1__ + gr * x1_s1, 5)
        
        x2_s2_ = max(x2_s2 - a_s2, 0)
        y1_s2  = min(2 * x2_s2_ + npp * 5, 10)
        x2_s2__ = max(x2_s2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
        y2_s2  = min(x2_s2__ + gr * x1_s2, 10)
        
        return ((y1_s1, y2_s1), (y1_s2, y2_s2))
    else:
        return rng.choice(X)
    
def P9(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x_s1, x_s2   = x
        x1_s1, x2_s1 = x_s1
        x1_s2, x2_s2 = x_s2
        a_s1, a_s2   = a 
        
        unif2 = np.random.uniform(0)

        x2_s1_ = max(x2_s1 - a_s1, 0)
        y1_s1  = min(2 * x2_s1_ + npp * 5, 5)
        x2_s1__ = max(x2_s1_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
        y2_s1  = min(x2_s1__ + gr * x1_s1, 5)
        
        x2_s2_ = max(x2_s2 - a_s2, 0)
        y1_s2  = min(2 * x2_s2_ + npp * 5, 10)
        x2_s2__ = max(x2_s2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
        y2_s2  = min(x2_s2__ + gr * x1_s2, 10)
        
        return ((y1_s1, y2_s1), (y1_s2, y2_s2))
    else:
        return rng.choice(X)

def P10(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x_s1, x_s2   = x
        x1_s1, x2_s1 = x_s1
        x1_s2, x2_s2 = x_s2
        a_s1, a_s2   = a 
        
        unif2 = np.random.uniform(0)

        x2_s1_ = max(x2_s1 - a_s1, 0)
        y1_s1  = min(2 * x2_s1_ + npp * 5, 5)
        x2_s1__ = max(x2_s1_ - ((unif2 > 1) * dr + (unif2 <= 1) * 1) * 5, 0)
        y2_s1  = min(x2_s1__ + gr * x1_s1, 5)
        
        x2_s2_ = max(x2_s2 - a_s2, 0)
        y1_s2  = min(2 * x2_s2_ + npp * 5, 10)
        x2_s2__ = max(x2_s2_ - ((unif2 > 1) * dr + (unif2 <= 1) * 1) * 5, 0)
        y2_s2  = min(x2_s2__ + gr * x1_s2, 10)
        
        return ((y1_s1, y2_s1), (y1_s2, y2_s2))
    else:
        return rng.choice(X)

def P11(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x_s1, x_s2   = x
        x1_s1, x2_s1 = x_s1
        x1_s2, x2_s2 = x_s2
        a_s1, a_s2   = a 
        
        unif2 = np.random.uniform(0)

        x2_s1_ = max(x2_s1 - a_s1, 0)
        y1_s1  = min(2 * x2_s1_ + npp * 5, 5)
        x2_s1__ = max(x2_s1_ - ((unif2 > 1) * dr + (unif2 <= 1) * 1) * 5, 0)
        y2_s1  = min(x2_s1__ + gr * x1_s1, 5)
        
        x2_s2_ = max(x2_s2 - a_s2, 0)
        y1_s2  = min(2 * x2_s2_ + npp * 5, 10)
        x2_s2__ = max(x2_s2_ - ((unif2 > 1) * dr + (unif2 <= 1) * 1) * 5, 0)
        y2_s2  = min(x2_s2__ + gr * x1_s2, 10)
        
        return ((y1_s1, y2_s1), (y1_s2, y2_s2))
    else:
        return rng.choice(X)
    
# CREATE THE PROBABILITY MEASURE OUT OF THE RANDOM VARIABLE
nr = 1_000
p3_ = np.zeros([len(X), len(A), len(X)])
p9_ = np.zeros([len(X), len(A), len(X)])
p10_ = np.zeros([len(X), len(A), len(X)])
p11_ = np.zeros([len(X), len(A), len(X)])
for n in range(nr):
    for x in X:
        for a in A:
            y3 = P3(x,a)
            x_3 = x_index(y3)
            p3_[x_index(x), a_index(a), x_3] += 1
            y9 = P9(x,a)
            x_9 = x_index(y9)
            p9_[x_index(x), a_index(a), x_9] += 1
            y10 = P10(x,a)
            x_10 = x_index(y10)
            p10_[x_index(x), a_index(a), x_10] += 1
            y11 = P11(x,a)
            x_11 = x_index(y11)
            p11_[x_index(x), a_index(a), x_11] += 1
p3_ = p3_/nr
p9_ = p9_/nr
p10_ = p10_/nr
p11_ = p11_/nr
def p3(x,a,y):
    return(p3_[x_index(x), a_index(a), x_index(y)])
def p9(x,a,y):
    return(p9_[x_index(x), a_index(a), x_index(y)])
def p10(x,a,y):
    return(p10_[x_index(x), a_index(a), x_index(y)])
def p11(x,a,y):
    return(p11_[x_index(x), a_index(a), x_index(y)])

In [ ]:
Nr_iter = 1_000_000

In [ ]:
Q_opt_robust, V = robust_q_learning_v2(X, A, r, np.array([P,P3,P9,P10,P11]), np.array([p,p3,p9,p10,p11]), alpha, x_0, k_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = Q_0_)

In [ ]:
Result = np.array([[a_opt(x, Q_opt_robust) for x in X]])
Result = Result[0]

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:

df = pd.DataFrame(Result.T)
df.columns = pd.MultiIndex.from_tuples([(str(x[0]).replace("[", "(").replace("]", ")"), str(x[1]).replace("[", "(").replace("]", ")")) for x in X]) #[str(x) for x in X]
df.index.name = "Species"
df.columns.names = ["A", "B"]
df

In [ ]:
plt.matshow(V, cmap='gray')
plt.colorbar()

Now lets assume that the two species doesn't necessarily have the same evolution parameters

In [ ]:
def P4(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x_s1, x_s2   = x
        x1_s1, x2_s1 = x_s1
        x1_s2, x2_s2 = x_s2
        a_s1, a_s2   = a 
        
        unif2 = np.random.uniform(0)

        x2_s1_ = max(x2_s1 - a_s1, 0)
        y1_s1  = min(rr * x2_s1_ + npp * 5, 5)
        y2_s1  = min(x2_s1_ - 5 * dr + gr * x1_s1, 5)
        
        x2_s2_ = max(x2_s2 - a_s2, 0)
        y1_s2  = min(2 * x2_s2_ + npp * 5, 10)
        x2_s2__ = max(x2_s2_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
        y2_s2  = min(x2_s2__ + gr * x1_s2, 10)
        
        return ((y1_s1, y2_s1), (y1_s2, y2_s2))
    else:
        return rng.choice(X)

def P5(x,a):
    unif      = np.random.uniform(0)
    if (unif > eps_greedy):
        x_s1, x_s2   = x
        x1_s1, x2_s1 = x_s1
        x1_s2, x2_s2 = x_s2
        a_s1, a_s2   = a 
        
        unif2 = np.random.uniform(0)

        x2_s1_ = max(x2_s1 - a_s1, 0)
        y1_s1  = min(2 * x2_s1_ + npp * 5, 5)
        x2_s1__ = max(x2_s1_ - ((unif2 > 0.5) * dr + (unif2 <= 0.5) * 1) * 5, 0)
        y2_s1  = min(x2_s1__ + gr * x1_s1, 5)
        
        x2_s2_ = max(x2_s2 - a_s2, 0)
        y1_s2  = min(rr * x2_s2_ + npp * 5, 10)
        y2_s2  = min(x2_s2_ - 5 * dr + gr * x1_s2, 10)
        
        return ((y1_s1, y2_s1), (y1_s2, y2_s2))
    else:
        return rng.choice(X)

    
# CREATE THE PROBABILITY MEASURE OUT OF THE RANDOM VARIABLE
nr = 1_000
p4_ = np.zeros([len(X), len(A), len(X)])
p5_ = np.zeros([len(X), len(A), len(X)])
for n in range(nr):
    for x in X:
        for a in A:
            y4 = P4(x,a)
            x_4 = x_index(y4)
            p4_[x_index(x), a_index(a), x_4] += 1
            y5 = P5(x,a)
            x_5 = x_index(y5)
            p5_[x_index(x), a_index(a), x_5] += 1
p4_ = p4_/nr
p5_ = p5_/nr
def p4(x,a,y):
    return(p4_[x_index(x), a_index(a), x_index(y)])
def p5(x,a,y):
    return(p5_[x_index(x), a_index(a), x_index(y)])

In [ ]:
Q_opt_robust, V = robust_q_learning_v2(X, A, r, np.array([P,P3,P4,P5]), np.array([p,p3,p4,p5]), alpha, x_0, k_0, eps_greedy, Nr_iter, gamma_t_tilde = lambda t: 1/(t+1), Q_0 = Q_0_)

In [ ]:
Result = np.array([[a_opt(x, Q_opt_robust) for x in X]])
Result = Result[0]

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:

df = pd.DataFrame(Result.T)
df.columns = pd.MultiIndex.from_tuples([(str(x[0]).replace("[", "(").replace("]", ")"), str(x[1]).replace("[", "(").replace("]", ")")) for x in X]) #[str(x) for x in X]
df.index.name = "Species"
df.columns.names = ["A", "B"]
df

In [ ]:
plt.matshow(V, cmap='gray')
plt.colorbar()